In [1]:
import pickle
from pytorch3d.structures import Meshes
import torch
import numpy as np
import math


import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1" 
# os.environ['NCCL_P2P_DISABLE'] = '1'
# os.environ['NCCL_IB_DISABLE'] = '1'
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# from training.volumetric_rendering.camera_utils2 import transform_to_view_space, project_to_clip_space

from collections import OrderedDict
from pytorch3d.io import load_obj, save_obj

In [5]:
_v_template, faces, aux = load_obj('assets/flame_uv_no_back_close_mouth_no_subdivision.obj', load_textures=False)

_faces = faces.verts_idx
_faces_uvs =  faces.textures_idx
_vertex_uvs = aux.verts_uvs

In [6]:
def generate_random_point_in_tetrahedron(vertices):
    # Generate random barycentric coordinates
    r1 = np.random.random()
    r2 = np.random.random()
    r3 = np.random.random()
    r4 = np.random.random()

    # print(r1, r2, r3)

    # Ensure that the sum of barycentric coordinates is <= 1
    total = r1 + r2 + r3 + r4
    r1 /= total
    r2 /= total
    r3 /= total
    r4 /= total

    # Calculate the random point using barycentric coordinates
    point = r1 * vertices[0] + r2 * vertices[1] + r3 * vertices[2] + r4 * vertices[3]

    return point, [r1, r2, r3, r4]


def generate_random_point_in_triangle(vertices):
    # Generate random barycentric coordinates
    r1 = np.random.random()
    r2 = np.random.random()
    r3 = np.random.random()
    # r4 = np.random.random()

    # print(r1, r2, r3)

    # Ensure that the sum of barycentric coordinates is <= 1
    total = r1 + r2 + r3 
    r1 /= total
    r2 /= total
    r3 /= total    

    # Calculate the random point using barycentric coordinates
    point = r1 * vertices[0] + r2 * vertices[1] + r3 * vertices[2] 

    return point, [r1, r2, r3]

class points:
    def __init__(self, coordinates= [], upper_traingle= [], lower_traingle= [], upper_uv= [], lower_uv= [], tet_bari= [], tex_up= [], tex_low= [], 
                 z_0 = [], z = [], z_1 = [], faces = []):
        self.coordinates = coordinates
        self.upper_traingle = upper_traingle
        self.lower_traingle = lower_traingle
        self.upper_uv = upper_uv
        self.lower_uv = lower_uv
        self.tet_bari = tet_bari
        self.tex_up = tex_up
        self.tex_low = tex_low
        self.z_0 = z_0
        self.z = z
        self.z_1 = z_1
        self.faces = faces


@torch.no_grad()
def generate_points(num_shells, faces, vertices, tex_coord, uv_faces):
   
        offset = torch.max(faces.reshape(-1)) 
        offset2 = torch.max(uv_faces.reshape(-1))
        orig_vertices = vertices.to(faces.device).reshape(-1, 3)
        vertices = orig_vertices

        # widen = np.linspace(1,1.5,num_shells-1)
        # vertices[:,0] = vertices[:,0] * 2
        vertices[:,0] = vertices[:,0] * 1.5
        tex_coord = tex_coord.to(faces.device).reshape(-1, 2).repeat(num_shells,1)
        print(tex_coord.shape)
        num1= 1
        num2=1

        # points_list = points()
     
        new_faces = []

        face_one = faces
        uv_faceone = uv_faces
        # self.upper = {}
        
        point_list = points()
        kk = 0 
        for i in range(1, num_shells):
            print(i)

            
            face_two = face_one + offset + 1
            uv_facetwo = uv_faceone + offset2 + 1
            kk = 0 
            # vertices = orig_vertices*widen[i-1]
            for face1, face2, uv_face1, uv_face2 in zip(face_one, face_two, uv_faceone, uv_facetwo):                            

                tetrahedra = []
                # print()
                tetrahedra.append([vertices[face1[0]], vertices[face1[1]], vertices[face1[2]]])
                # tetrahedra.append([vertices[face2[0]],vertices[face2[1]], vertices[face2[2]]])

                for jj in range(num1):
                
                    for ii in range(num2):
                        point, uvw_ = generate_random_point_in_triangle(tetrahedra[jj])
                        # if jj == 0 :
                            # print(tex_coord,uv_face1,uvw_)
                        texture_interp_up = tex_coord[uv_face1[0]] * uvw_[0] + tex_coord[uv_face1[1]] * uvw_[1] + tex_coord[uv_face1[2]] * uvw_[2]
                        # else:
                            # texture_interp_up = tex_coord[uv_face2[0]] * uvw_[0] + tex_coord[uv_face2[1]] * uvw_[1] + tex_coord[uv_face2[2]] * uvw_[2]
                       
                        # print(uvw_, texture_interp_up)
                        point_list.coordinates.append(point)
                        # point_list.upper_traingle.append(kk)    
                        # point_list.lower_traingle.append(kk + 1)
                        point_list.upper_uv.append(torch.tensor(uvw_))
                        point_list.faces.append(face1)
                        point_list.tex_up.append(texture_interp_up)
                num1= 1
                num2=1
                  
                kk+=1

            face_one = face_two
            uv_faceone = uv_facetwo
        
        # new_faces = torch.stack(new_faces)
        point_list.tex_up = torch.stack(point_list.tex_up)
        # print(point_list.upper_uv)
        point_list.upper_uv = torch.stack(point_list.upper_uv)
        point_list.faces = torch.stack(point_list.faces)
        point_list.coordinates = torch.stack(point_list.coordinates)
      
        return point_list, vertices


@torch.no_grad()
def settings_(vertices, faces, uv_faces, num_shells, tex_coord, normal_scale=0.04, disable_small = False):
    faces = torch.tensor(faces.astype(np.int64)).to(vertices.device)
    print(faces[None].repeat(vertices.shape[0], 1, 1).shape)
    print('NORMAL EXTENSION')
    mesh = Meshes(
        verts=vertices, faces=faces[None].repeat(vertices.shape[0], 1, 1)
    )
    orig_vertex_ = vertices.clone()
    # bs, n, 3
    vertex_normal = mesh.verts_normals_padded()

    linscale = torch.linspace(normal_scale * 2, -normal_scale*3, num_shells)

    # linscale = torch.linspace(normal_scale * 3, -normal_scale*2, num_shells)


    # linscale = torch.linspace(normal_scale * 2, -normal_scale*4, num_shells)
    # linscale = torch.linspace(0, -normal_scale, num_shells)
    # kfs
    if disable_small:  # disable small meshes (meshes that are inside SMPL)
        linscale = torch.linspace(0, 0, num_shells)
    linscale = linscale.to(vertices.device).to(vertices.dtype)
    vertices = (
        vertices[:, None]
        + linscale.reshape(1, num_shells, 1, 1) * vertex_normal[:, None]
    )
    
    points_list, new_vertices  = generate_points(num_shells, faces, vertices, tex_coord, uv_faces)
    # _uv_faces = convert_to_tetrahedron(2, uv_faces)

    return points_list, new_vertices


points_list, vertices__ = settings_((_v_template*5
                      ).unsqueeze(0), _faces.numpy(), _faces_uvs,5, _vertex_uvs, normal_scale=0.04, disable_small = False)



torch.Size([1, 7004, 3])
NORMAL EXTENSION
torch.Size([18300, 2])
1
2
3
4


In [7]:

print('Points:', points_list.coordinates.shape)
print('Faces_per_point:', points_list.faces.shape)
print('uvw_per_point', points_list.upper_uv.shape)
print('texture_ccord_per_point', points_list.tex_up.shape)
print('vertices_flattened', vertices__.shape)


# Given the face indices, find the vertices of the triangles
def indexed_tensor(faces, vertices):
    # Extract the indices for each vertex in a face
    indices = faces.view(-1)
    
    # Use torch.index_select to get the indexed tensor
    indexed_tensor = torch.index_select(vertices, dim=0, index=indices)
    
    # Reshape the indexed tensor to match the original faces shape
    indexed_tensor = indexed_tensor.view(faces.shape[0], faces.shape[1], -1)

    return indexed_tensor

# Find the points using baricentric interpolation
def combine_vertices(indexed_tensor, weights):
    # Make sure the shapes are compatible
    assert indexed_tensor.shape[0] == weights.shape[0]
    
    # Expand the weights tensor to have the same shape as indexed_tensor
    expanded_weights = weights.unsqueeze(2)
    
    # Multiply the indexed tensor by the expanded weights and sum along the second dimension
    combined_tensor = torch.sum(indexed_tensor * expanded_weights, dim=1)
    
    return combined_tensor

# This is where the deformation takes place (per vertex)
vertices__ = vertices__ + torch.randn(vertices__.shape)

#These are the new triangles where the faces are the same by the vertex values are different
indexed_tensor_ = indexed_tensor(points_list.faces, vertices__)

# Do the interpolation based on the saved uvw to generate dispalcement in the points
new_coordinates  = combine_vertices(indexed_tensor_, points_list.upper_uv)

# The points are different from original points (if the deformation is not added then points should be the same)
print(torch.unique(points_list.coordinates - new_coordinates))


Points: torch.Size([28016, 3])
Faces_per_point: torch.Size([28016, 3])
uvw_per_point torch.Size([28016, 3])
texture_ccord_per_point torch.Size([28016, 2])
vertices_flattened torch.Size([17765, 3])
tensor([-3.0004, -2.7329, -2.7221,  ...,  3.0149,  3.0213,  3.0613])
